In [1]:
!pwd

/ekaterina/work/src/lca/lca


In [10]:
'''
Assign the log file name.
'''

clustering_file = '/ekaterina/work/src/lca/lca/tmp/zebra_drone/zebra_drone_0088_right_filtered_chuck/clustering.json'
node2uuid_file = '/ekaterina/work/src/lca/lca/tmp/zebra_drone/zebra_drone_0088_right_filtered_chuck/node2uuid_file.json'
anno_file = '/ekaterina/work/src/lca/lca/data/drone/chuck_ca_filtered_final_combined_output_dji_0088_right_filtered.json'

In [11]:
import pandas as pd
import json

def load_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

data = load_json(anno_file)

dfc = pd.DataFrame(data['categories'])
dfa = pd.DataFrame(data['annotations'])
dfi = pd.DataFrame(data['images'])

df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = df.merge(dfc, left_on='id', right_on='id')

# df = df[df['annotations_census'] == True]
df = df[df['viewpoint'] == 'right']

counts = df.groupby('viewpoint').size()
print(counts)


viewpoint
right    155
dtype: int64


In [12]:
clusters = load_json(clustering_file)
node2uuid = load_json(node2uuid_file)


uuid_to_cluster = {}


for cluster_id, nodes in clusters.items():
    for node in nodes:
        uuid = node2uuid.get(str(node))  # Get UUID from node2uuid
        if uuid:
            uuid_to_cluster[uuid] = cluster_id  # Map UUID to cluster ID

print(uuid_to_cluster)
# print(node2uuid)

{'067de032-c66f-4aca-a84d-7579f8955ed6': 'tc133', '5c70a527-a694-4dd5-b660-43a3bd7ac8ef': 'tc165', '0c1cbead-dbc3-430a-819d-db2f97386ff8': 'tc165', 'e05dac82-0326-4705-91e5-21ed761a6e14': 'tc165', 'f27a8c65-d8ae-495c-b87a-edeb57ee7e7a': 'tc165', '62caa700-2864-4eaa-8b43-acc890c6abef': 'tc165', '992cdb9c-dec5-4203-8b10-3454443d58f7': 'tc165', '6d332b40-8499-4e9a-8e6f-94d9391c0ada': 'tc165', '94e7babe-eaca-46e0-a48f-45e8c6552cd7': 'tc165', 'f4b9ac39-3f08-4fd1-802e-658903afcba6': 'tc165', '99d9213c-da0f-4ebd-bb8b-5ec41bef1039': 'tc165', '85ab13cc-01cf-4209-9450-bdc9574c546d': 'tc165', '7a444acd-be31-4e6c-8b2a-7d9656bf07a0': 'tc165', '7a3bca4b-71b2-4006-8971-dccff3593cd9': 'tc272', 'eac562f0-8d5b-48d3-b735-06421806bcaf': 'tc272', '3e677d7e-eb52-4662-9702-406c6793fcc5': 'tc272', '234a5965-be15-424c-8bbc-25b71599dc12': 'tc272', 'fb7cbe3a-aaa2-4275-8f77-14783b7977a0': 'tc272', '15e93d5b-41a1-44c0-9dea-f41a9155da1b': 'tc272', '78d42940-4192-4733-addd-c8a657f694b8': 'tc272', '80a973cf-7511-4b25

In [13]:
import random
cluster_ids = list(clusters.keys())

# Function to get cluster ID or assign a random cluster if missing
def get_cluster_or_random(uuid):
    return uuid_to_cluster.get(uuid, random.choice(cluster_ids))

# Apply the function to map or assign random values for missing mappings
df['uuid'] = df['uuid_x']
# df['LCA_clustering_id'] = df['uuid'].apply(get_cluster_or_random)

df['LCA_clustering_id'] = df['uuid'].map(uuid_to_cluster)

In [14]:
import json
output_path = '/ekaterina/work/src/lca/lca/data/drone/chuck_ca_filtered_final_combined_output_dji_0088_right_filtered_LCA.csv.json'

df_annotations_fields = ['uuid', 'image_uuid', 'bbox', 'viewpoint', 'tracking_id', 'individual_id', 'confidence','detection_class','species', 'CA_score', 'category_id', 'LCA_clustering_id']
df_annotations = df[df_annotations_fields]
# df_annotations = df_annotations.rename(columns={'uuid_x': 'uuid'})


df_images_fields = ['image_uuid', 'file_name']
df_images_fields= df.columns.intersection(df_images_fields)
df_images = df[df_images_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_images = df_images.rename(columns={'image_uuid': 'uuid'})


df_categories_fields = ['category_id', 'species']
df_categories = df[df_categories_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_categories = df_categories.rename(columns={'category_id': 'id'})

result_dict = {
                'categories': df_categories.to_dict(orient='records'),
                'images': df_images.to_dict(orient='records'),
                'annotations': df_annotations.to_dict(orient='records')
                }

with open(output_path, 'w') as f:
    json.dump(result_dict, f, indent=4)
    print('Data is saved to:', output_path)


Data is saved to: /ekaterina/work/src/lca/lca/data/drone/chuck_ca_filtered_final_combined_output_dji_0088_right_filtered_LCA.csv.json


13578


Histogram of weights
-100, -90: 0
-90, -80: 0
-80, -70: 0
-70, -60: 20
-60, -50: 581
-50, -40: 2899
-40, -30: 3824
-30, -20: 1881
-20, -10: 1279
-10, 0: 1489
0, 10: 476
10, 20: 332
20, 30: 163
30, 40: 123
40, 50: 110
50, 60: 81
60, 70: 76
70, 80: 55
80, 90: 41
90, 100: 148
Num negative weights 11973
Num positive weights 1605
